## Some ideas for a numerical solution to the SPACE model: Stream Power with Alluvium Conservation and Entrainment
Basic equation set so far, for 1D. Sediment flux downstream:

$$\frac{dq_s}{dx} = K_s q S (1-\exp (-H/H_* ) ) + (1-F_f) K_r q S \exp (-H/H_*) -Vq_s/q$$

Rate of change of alluv thickness:

$$(1-\phi ) \frac{\partial H}{\partial t} = V q_s/q - K_s q S (1-\exp (-H/H_* ) )$$

Rate of change of rock elev:

$$\frac{\partial R}{\partial t} = U - K_r q S \exp (-H/H_* )$$

Local analytical solution for sediment flux within a cell:

$$q_s = \left( \frac{E_s+(1-F_f)E_r}{V/q} \right) \left( 1-\exp (-Vx/q) \right) + q_{s0} \exp (-Vx/q)$$

Try a little 1D version with some initial slope $S_0$:

In [32]:
import numpy as np

num_nodes = 5
dx = 1.0
S0 = 0.01
q = 1.0
V = 0.001
Ff = 0.5
Ks = 1.0
Kr = 0.001
Hstar = 1.0
Voverq = V / q
dt = 1.0
U = 1.0e-6

# handy array index
upper = np.arange(num_nodes - 1)

# arrays for qs, x, H, R, and z
qs = np.zeros(num_nodes)
qs_in = np.zeros(num_nodes)
x = dx * np.arange(0, num_nodes)
H = np.zeros(num_nodes)
R = np.zeros(num_nodes)
z = S0 * (num_nodes - 1) * dx - S0 * x
R[:] = z
z

array([ 0.04,  0.03,  0.02,  0.01,  0.  ])

Calculate $E_s$ and $E_r$ for all nodes:

In [33]:
S = (z[upper] - z[1:]) / dx
Es = Ks * q * S * (1.0 - np.exp(-H[upper]/ Hstar))
Er = Kr * q * S * np.exp(-H[upper] / Hstar)
Er

array([  1.00000000e-05,   1.00000000e-05,   1.00000000e-05,
         1.00000000e-05])

Now loop from upstream to downstream calculating flux:

In [34]:
for i in range(num_nodes - 1):
    qs[i] = ((Es[i] + (1-Ff) * Er[i]) / Voverq) * (1.0 - np.exp(-dx * Voverq)) + qs_in[i] * np.exp(-dx * Voverq)
    qs_in[i+1] = qs[i]
qs

array([  4.99750083e-06,   9.99000666e-06,   1.49775225e-05,
         1.99600533e-05,   0.00000000e+00])

Now do a forward Euler solution for $H$:

In [35]:
H[upper] += dt * (qs[upper] * Voverq) - Ks * q * S * (1.0 - np.exp(-H[upper] / Hstar))
H

array([  4.99750083e-09,   9.99000666e-09,   1.49775225e-08,
         1.99600533e-08,   0.00000000e+00])

Now, rock erosion/uplift and elevations:

In [40]:
R[upper] += dt * (U - Kr * q * S * np.exp(-H[upper] / Hstar))
z = R + H
z

array([ 0.039955  ,  0.02995501,  0.01995501,  0.00995502,  0.        ])